In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pypandoc
!pip install transformers
!pip install docx2txt
!pip install python-docx 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=512ecbfc7202e5d54803b00129a6ed4136ce4a2a230a23fd78d7feef55edb986
  Stored in directory: /root/.cache/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
!pip install python-docx
!pip install lxml==4.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.2
    Uninstalling lxml-4.9.2:
      Successfully uninstalled lxml-4.9.2


In [3]:
!pip install bayoo-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 kB 4.5 MB/s eta 0:00:00


In [4]:
# Required import
import nltk
from nltk.corpus import wordnet

import docx2txt
import spacy
import re
from docx import Document

from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

import docx
from docx.enum.text import WD_COLOR_INDEX
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pypandoc
import plotly.figure_factory as ff
import seaborn as sns
import tensorflow
from nltk.corpus import stopwords
from plotly.offline import iplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from transformers import BertConfig, BertTokenizerFast, TFBertModel

In [5]:
! pip show lxml

Name: lxml
Version: 4.9.1
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: /usr/local/lib/python3.9/dist-packages
Requires: 
Required-by: bayoo-docx, nbconvert, pandas-datareader, python-docx, yfinance


In [6]:
start_time = time.time()

Find Synonyms of words in dealbreaker words list

In [7]:
# create function to find synonyms from the dealbreaker words list

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

In [8]:
# Already existed words in Parties 

Parties = ["Unilateral", "One-Way", "One Way", "company (\"Disclosing Party\")", "company (\"Discloser\")", "corporation (\"Disclosing Party\")", "corporation (\"Discloser\")",  "LLC (\"Disclosing Party\")", "LLC (\"Discloser\")",  "Inc. (\"Disclosing Party\")", "Inc. (\"Discloser\")",  "Incorporated (\"Disclosing Party\")", "Incorporated (\"Discloser\")", "Co. (\"Disclosing Party\")", "Co. (\"Discloser\")"]

# No change because these words really do not have synonyms

In [9]:
# Already existed words in Residuals/ Memories

Residuals = ["residual", "residuals", "memories", "unaided", "memory"]

# find synonyms
synonyms_residuals = []
for word in Residuals:
    synonyms_residuals.append(get_synonyms(word))

## synonyms_residuals

# since each word in Residuals will generate a list of its synonyms,
# # hence, combine these sublists into one list
combined_synonyms_residuals = []
for sublst in synonyms_residuals:
    combined_synonyms_residuals.extend(sublst)

# clean the list to only pick up unique words, since some words' synonyms may be repeated
unique_synonyms_residuals = list(set(combined_synonyms_residuals))


# Notice: 
# There are TWO lists of words of Residuals now:
# Only is only about Residuals Dealbreaker words
# Second one is about synonyms of Residuals Dealbreaker words.



In [10]:
# Arealdy existed words in Limitation of Liability

Limitation = ["Limitation of Liability", "under no circumstances", "shall be limited", "special, incidental, indirect or consequential damages", "punitive","exemplary","consequential","indirect","incidental"]
words_in_limitation = ["punitive","exemplary","consequential","indirect","incidental"]

# find synonyms
synonyms_limitation = []
for word in words_in_limitation:
    synonyms_limitation.append(get_synonyms(word))

## synonyms_limitation

# combined sublists into one list
combined_synonyms_limitation = []
for sublst in synonyms_limitation:
    combined_synonyms_limitation.extend(sublst)

# unique words, some have repeated words
unique_synonyms_limitation = list(set(combined_synonyms_limitation))

# Notice: 
# There are TWO lists of words of Limitation of Liability now:
# Only is only about Limitation of Liability Dealbreaker words
# Second one is about synonyms of Limitation of Liability Dealbreaker words.

In [11]:
# Already existed words in Non-competition

Noncompetition = ["compete", "competition", "non-compete", "non-competition", "non compete", "non competition" ]

# find synonyms
synonyms_noncompetition = []
for word in Noncompetition:
    synonyms_noncompetition.append(get_synonyms(word))

## synonyms_noncompetition

# combined sublists into one list
combined_synonyms_noncompetition = []
for sublst in synonyms_noncompetition:
    combined_synonyms_noncompetition.extend(sublst)

# unique words, some have repeated words
unique_synonyms_noncompetition = list(set(combined_synonyms_noncompetition))



# Notice: 
# There are TWO lists of words of Noncompetition now:
# Only is only about Residuals Noncompetition words
# Second one is about synonyms of Noncompetition Dealbreaker words.

In [12]:
# Already existed words in Non-solicitation

Nonsolicitation = ["non-solicitation", "solicit", "non-solicit", "non-servicing",   "nonsolicitation", "nonsolicit", "nonservicing",  "solicit",  "non solicitation", "non solicit", "non servicing",  "no solicit", "no-solicit"]

# find synonyms
synonyms_nonsolicitation = []
for word in Nonsolicitation:
    synonyms_nonsolicitation.append(get_synonyms(word))

## synonyms_nonsolicitation

# combined sublists into one list
combined_synonyms_nonsolicitation = []
for sublst in synonyms_nonsolicitation:
    combined_synonyms_nonsolicitation.extend(sublst)

# unique words, some have repeated words
unique_synonyms_nonsolicitation = list(set(combined_synonyms_nonsolicitation))


# Notice: 
# There are TWO lists of words of Nonsolicitation now:
# Only is only about Residuals Dealbreaker words
# Second one is about synonyms of Residuals Dealbreaker words.

In [13]:
# Already existed words in Indemnification

Indemnification = ["indemnification", "indemnity", "hold-harmless", "hold harmless", "indemnify", "indemnified", "indemnifying", "defend"]

# find synonyms
synonyms_indemnification = []
for word in Indemnification:
    synonyms_indemnification.append(get_synonyms(word))

## synonyms_indemnification

# combined sublists into one list
combined_synonyms_indemnification = []
for sublst in synonyms_indemnification:
    combined_synonyms_indemnification.extend(sublst)

# unique words, some have repeated words
unique_synonyms_indemnification = list(set(combined_synonyms_indemnification))

# Notice: 
# There are TWO lists of words of Indemnification now:
# Only is only about Indemnification Dealbreaker words
# Second one is about synonyms of Indemnification Dealbreaker words.

In [14]:
# Already existed words in Governing Law/ Jurisdiction

# No change, these words are only country names.

Governing = ["Texas", "Italy", "Italian"]

In [15]:
# Already existed words in Exceptions

# No changes, they are not words but lines.
Exceptions =  ["was communicated by Disclosing Party to an unaffiliated third party free of any obligation of confidence",   "was communicated by Disclosing Party to a third party free of any obligation of confidence", "was communicated by Disclosing Party to  a third party without an obligation of confidence", "was disclosed by Disclosing Party to  a third party without an obligation of confidence", " was disclosed by Discloser to a third party without an obligation of confidentiality"]


In [16]:
# Already existed words for Representatives

# Different from the above, this part is for existence, if exist, yes, if not exist, output a warning sign about "Missing Representatives"

Representatives = ["responsible", "liable"]

# find synonyms
synonyms_representatives = []
for word in Representatives:
    synonyms_representatives.append(get_synonyms(word))

## synonyms_representatives

# combined sublists into one list
combined_synonyms_representatives = []
for sublst in synonyms_representatives:
    combined_synonyms_representatives.extend(sublst)

# unique words, some have repeated words
unique_synonyms_representatives = list(set(combined_synonyms_representatives))


# Notice: 
# There are TWO lists of words of Representatives now:
# Only is only about Representatives Dealbreaker words
# Second one is about synonyms of Representatives Dealbreaker words.

In [17]:
# Already existed words for Remedies

# Same as the last one, this part is for existence, if exist, yes, if not exist, output a warning sign about "Missing Remedies"
Remedies = ["injunction", "injunctive","equitable"]

# find synonyms
synonyms_remedies = []
for word in Remedies:
    synonyms_remedies.append(get_synonyms(word))

## synonyms_remedies

# combined sublists into one list
combined_synonyms_remedies = []
for sublst in synonyms_remedies:
    combined_synonyms_remedies.extend(sublst)

# unique words, some have repeated words
unique_synonyms_remedies = list(set(combined_synonyms_remedies))


# Notice: 
# There are TWO lists of words of Remedies now:
# Only is only about Remedies Dealbreaker words
# Second one is about synonyms of Remedies Dealbreaker words.

In [18]:
# Already existed words for Privacy / Personal Information
Privacy = ["Personal Data", "Personal Information","PII", "GDPR", "CCPA", "CPRA", "Privacy"]
words_in_privacy = ["Privacy"]

# find synonyms
synonyms_privacy = []
for word in words_in_privacy:
    synonyms_privacy.append(get_synonyms(word))

## synonyms_privacy

# combined sublists into one list
combined_synonyms_privacy = []
for sublst in synonyms_privacy:
    combined_synonyms_privacy.extend(sublst)

# unique words, some have repeated words
unique_synonyms_privacy = list(set(combined_synonyms_privacy))

# Notice: 
# There are TWO lists of words of Privacy now:
# Only is only about Privacy Dealbreaker words
# Second one is about synonyms of Privacy Dealbreaker words.


Test Trials

In [19]:
# Test 3

##           ###           ##  EEEEEEEEEEEEEEEE      III
 ##         ## ##         ##   E                     III
  ##       ##   ##       ##    E                     III
   ##     ##     ##     ##     EEEEEEEEEEEEEEE       III
    ##   ##       ##   ##      E                     III
     ## ##         ## ##       E                     III
      ##            ##         EEEEEEEEEEEEEEEE      III
def ssp():
    
    # Recreate the exact same model, including its weights and the optimizer
    bert_model = tensorflow.keras.models.load_model('/content/drive/MyDrive/Final_pipeline/my_model.h5')

    # # Show the model architecture
    # bert_model.summary()
    import spacy
    import re
    import nltk
    #en_core_web_sm: this package must be the newest version, currently 3.5.0   (by 2023.04.02)
    !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0.tar.gz --quiet
    import en_core_web_sm
    #open text file in read mode
    docxFilename = "/content/drive/MyDrive/Final_pipeline/NDA.docx"
    output = pypandoc.convert_file([docxFilename], 'plain', outputfile="/content/drive/MyDrive/Final_pipeline/text_file.txt")
    text_file = open("/content/drive/MyDrive/Final_pipeline/text_file.txt", "r")
    
    #read whole file to a string
    data = text_file.read()

    #close file
    text_file.close()
    #def a function which can split the doc into sentences
    def sentence_split(data):
        # nlp = spacy.load("en_core_web_sm")
        nlp = en_core_web_sm.load()
        
        #take unicode string  
        sentences = nlp(data)


        sentencess = list(sentences.sents)

        separated_sentences = []

        for i in range(len(sentencess)):
            sentence = sentencess[i].text
            #sentence_separation = re.split('\t|\n|\n\t', sentence)
            sentence = sentence.replace('\n', '')
            sentence = sentence.replace('\t', '')
            sentence = sentence.replace('\ufeff', '')
            sentence = sentence.replace('  ', '')
            sentence = sentence.replace('   ', '') #there are always multiple spaces, change them to single space universally
            sentences = re.split(r'\.(?=\s|$)', sentence)
            separated_sentences += sentences


        final_separated_sentences = [x for x in separated_sentences if x != '']
        return final_separated_sentences


    splitted = sentence_split(data)

    #now define a function seperating the document into list of paragraphs

    def par(splitted):
        ind = 0
        paragraphs = []
        splitting_index = []

        #get the index of splitting
        for i in range(len(splitted)):
            if len(splitted[i].split()) < 5:
                splitting_index.append(i)
        
        for j in range(len(splitting_index)):
            temp_para = ""
            while ind < splitting_index[j]:
                temp_para += splitted[ind]
                ind += 1
            paragraphs.append(temp_para)
        return [p for p in paragraphs if len(p.split()) > 5]

    paragraphs = par(splitted)
    #paragraphs

    # Name of the BERT model to use
    model_name = 'bert-base-uncased'

    # Max length of tokens
    max_length = 100

    # Load transformers config and set output_hidden_states to False
    config = BertConfig.from_pretrained(model_name)
    config.output_hidden_states = False

    # Load BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)


    test_x = tokenizer(
        text=par(splitted),
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding=True, 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = False,
        verbose = True)

    #test_x
    #predict the possible value 
    bert_result = bert_model.predict(test_x['input_ids'])
    #define a function, which can return the top 3 potential categories of a paragraph
    #this method would return a dictionary: key is each paragraph, the corresponding values a the top 3 potential categories
    target_names = ['DEF', 'EXP', 'GOV', 'REM', 'RIG', 'TER', 'WAR']

    def par_cat(bert_result, paragraphs, target_names, topN):
        result = {}
        for i in range(len(bert_result["CAT"])):
            val = np.argsort(bert_result["CAT"][i])[::-1][:topN]
            result[paragraphs[i]] = [target_names[v] for v in val]
        return result

    return par_cat(bert_result, paragraphs, target_names, 3)


# don't change this name.
original_input_dictionary = ssp()



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


1/1 [==============================] - 11s 11s/step


In [20]:
paragraphs = list(original_input_dictionary.keys())

In [21]:
# Segment paragraph into sentences. 
nlp = spacy.load("en_core_web_sm")

sentences_in_paragraphs = []  

# take unicode string  
for i in range(len(paragraphs)):
    sentences_in_paragraph = nlp(paragraphs[i])

    sentencess = list(sentences_in_paragraph.sents)

    separated_sentences = []
    # list all sentences
    for j in range(len(sentencess)):
        sentence = sentencess[j].text
    
    
        sentence = sentence.replace('\n\n', ' ')
        sentences = re.split(r'\.(?=\s|$)', sentence)


        separated_sentences += sentences

    final_separated_sentences = [x for x in separated_sentences if x != '']

    sentences_in_paragraphs.append(final_separated_sentences)

## sentences_in_paragraphs

Create dictionary for dealbreaker catergories and words with each category

In [22]:
# Recall in previous section, we have already find synonyms of dealbreaker words and list them. 

## Parties
## unique_synonyms_residuals
## unique_synonyms_limitation
## unique_synonyms_noncompetition
## unique_synonyms_nonsolicitation
## unique_synonyms_indemnification
## Governing
## Exceptions
## unique_synonyms_representatives
## unique_synonyms_remedies
## unique_synonyms_privacy

In [23]:
# create a dictionary of sensitive words, grouped them by different categories. 

dealbreaker_words = {
    "Parties":Parties,
     "Residuals/Memories": Residuals,
     "Limitation of Liability": Limitation,
     "Non-competition": Noncompetition,
     "Non-solicitation":Nonsolicitation,
     "Indemnification": Indemnification,
     "Governing Law/Jurisdiction":Governing,
     "Exceptions":Exceptions,
     "Privacy": Privacy
}

# The reason why not including remedies and representatives is due to the reason that
# their search methods are for existence (recall if they exist, then everything is ok, but if not exist, Warning)

In [24]:
# create a dictionary of synonyms of sensitive words, grouped them by different categories. 

synonyms_dealbreaker_words = {
     "Synonyms of Residuals/Memories": unique_synonyms_residuals,
     "Synonyms of Limitation of Liability": unique_synonyms_limitation,
     "Synonyms of Non-competition": unique_synonyms_noncompetition,
     "Synonyms of Non-solicitation":unique_synonyms_nonsolicitation,
     "Synonyms of Indemnification": unique_synonyms_indemnification,
     "Synonyms of Privacy": unique_synonyms_privacy
}

# The reason why not including remedies and representatives is due to the reason that
# their search methods are for existence (recall if they exist, then everything is ok, but if not exist, Warning)

In [25]:
update_dealbreaker_words = {
    "DEF": Parties ,
    "RIG": Limitation,
    "EXP": Exceptions,
    "WAR": Parties + Residuals + Limitation + Noncompetition + Nonsolicitation + Indemnification + Governing + Exceptions + Privacy,
    "GOV": Governing,
    "REM": "1",
    "TER": Parties + Residuals + Limitation + Noncompetition + Nonsolicitation + Indemnification + Governing + Exceptions + Privacy
}

In [26]:
update_synonyms_dealbreaker_words = {
    "RIG": unique_synonyms_limitation,
    "WAR": unique_synonyms_residuals + unique_synonyms_limitation + unique_synonyms_noncompetition + unique_synonyms_nonsolicitation + unique_synonyms_indemnification  + unique_synonyms_privacy,
    "REM": "2",
    "TER": unique_synonyms_residuals + unique_synonyms_limitation + unique_synonyms_noncompetition + unique_synonyms_nonsolicitation + unique_synonyms_indemnification  + unique_synonyms_privacy
}

Find each paragraph's dealbreaker words

In [27]:
def find_specific_dealbreakers(input_keys, input_dictionary):
    my_dict = {"new key":[]}
    for i in range(len(list(input_dictionary.keys()))):
        for j in range(len(input_keys)):
            if input_keys[j] in list(input_dictionary.keys())[i]: 
                my_dict["new key"] += input_dictionary[list(input_dictionary.keys())[i]]
    return my_dict

First about dealbreaker words

In [28]:
combined_check_list = []

for j in range(len(paragraphs)):
    #check_list = []

    new_dict = find_specific_dealbreakers(list(original_input_dictionary.values())[j], update_dealbreaker_words)

    for i in range(len(sentences_in_paragraphs[j])):
        sentence = sentences_in_paragraphs[j][i]
        # Iterate through the sensitive words grouped by category
        for category, words in new_dict.items():
            # Use regular expressions to search for the words in the sentence
            for word in words:
                matches = re.finditer(r'\b'+word+r'\b', sentence, re.IGNORECASE)
                for match in matches:
                    combined_check_list.append(match.group())


## combined_check_list

In [29]:
combined_check_list_syn = []

for j in range(len(paragraphs)):
    new_dict = find_specific_dealbreakers(list(original_input_dictionary.values())[j], update_synonyms_dealbreaker_words)

    for i in range(len(sentences_in_paragraphs[j])):
        sentence = sentences_in_paragraphs[j][i]
        # Iterate through the sensitive words grouped by category
        for category, words in new_dict.items():
            # Use regular expressions to search for the words in the sentence
            for word in words:
                matches = re.finditer(r'\b'+word+r'\b', sentence, re.IGNORECASE)
                for match in matches:
                    combined_check_list_syn.append(match.group())

## combined_check_list_syn

In [30]:
# Clean it for unique words, Upper or lower case matters!

combined_check_list = list(set(combined_check_list))
combined_check_list_syn = list(set(combined_check_list_syn))

## combined_check_list
## combined_check_list_syn

Then, about representative and rememdies. <br>
Here we need to find out whether they exist or not.

In [31]:
# Representatives
representative_warning = True

for j in range(len(original_input_dictionary)):

    for i in range(len(sentences_in_paragraphs[j])):
        sentence = sentences_in_paragraphs[j][i]
        
        for word in Representatives:
            
            if word in sentence:
                representative_warning = False
                ## print(f"Found '{word}' in sentence: {sentence}")
                break
        
        else:
            continue

        break
    


##representative_warning

In [32]:
# Remedies
remedies_warning = True

for j in range(len(paragraphs)):

    for i in range(len(sentences_in_paragraphs[j])):
        sentence = sentences_in_paragraphs[j][i]
        
        for word in Remedies:
            if word in sentence:
                remedies_warning = False
                ## print(f"Found '{word}' in sentence: {sentence}")
                break
        else:
            continue

        break


In [33]:
# Representatives Synonyms
syn_representative_warning = True

for j in range(len(paragraphs)):

    for i in range(len(sentences_in_paragraphs[j])):
        sentence = sentences_in_paragraphs[j][i]

        for word in unique_synonyms_representatives:
            if word in sentence:
                syn_representative_warning = False
                ## print(f"Found '{word}' in sentence: {sentence}")
                break
        else:
            continue

        break



if representative_warning == False:
    syn_representative_warning == False

In [34]:
# Synonyms Remedies
syn_remedies_warning = True

for j in range(len(paragraphs)):

    for i in range(len(sentences_in_paragraphs[j])):
        sentence = sentences_in_paragraphs[j][i]

        for word in unique_synonyms_remedies:
            if word in sentence:
                syn_remedies_warning = False
                ## print(f"Found '{word}' in sentence: {sentence}")
                break
        else:
            continue

        break



if remedies_warning == False:
    syn_remedies_warning == False

Export Stage

Trial

In [35]:
##           ###           ##  EEEEEEEEEEEEEEEE      III
 ##         ## ##         ##   E                     III
  ##       ##   ##       ##    E                     III
   ##     ##     ##     ##     EEEEEEEEEEEEEEE       III
    ##   ##       ##   ##      E                     III
     ## ##         ## ##       E                     III
      ##            ##         EEEEEEEEEEEEEEEE      III

In [36]:
# Import document again!
# change the path to your current path
doc=docx.Document('/content/drive/MyDrive/Final_pipeline/NDA.docx')

In [37]:
def find_key_by_value(value, dictionary):
    keys = []
    for key, val in dictionary.items():
        if isinstance(val, list):
            if any(v.lower() == value.lower() for v in val):
                keys.append(key)
        elif val.lower() == value.lower():
            keys.append(key)
    return keys if keys else None  # if value is not found in any key

Giving comment on dealbreaker words

In [39]:
def split_text(text, word):
    pattern = re.compile(r'([\S\s]*)(\b{})([\S\s]*)'.format(word))
    match = pattern.search(text)
    if match:
        return match.groups()
    return None

def split_Runs(doc,word):
    for p in doc.paragraphs:
        if p.text.find(word) != -1:
            virtualRuns=p.runs
            p.text = ""
            for r in virtualRuns:
                if r.text.find(word) != -1:
                    before, word, after = split_text(r.text, word)
                    p.add_run(before)
                    p.add_run()
                    p.add_run(word)
                    p.add_run(after)
                else:
                    p.add_run(r.text)
    return doc



def style_Token2(doc,word,list1, dictionary1,dictionary2, comment=True):
    for p in doc.paragraphs:
        for i,r in enumerate(p.runs):
          if word in list1:
            if (p.text.find(word + ",")) != -1 or (p.text.find(word + "."))  != -1 or (p.text.find(word + " ")) != -1:
                if p.runs[i].text.find(word) != -1:
                    p.runs[i].font.highlight_color = WD_COLOR_INDEX.RED
                    if comment:
                        corresponded_key = find_key_by_value(word,dictionary1)
                        p.runs[i-1].add_comment(f'Wanring: \"{word}\" is classified as a category {corresponded_key} dealbreaker word.\nStrongly Advice its Immediate Removal',author="DG1")
                        
            else:
              if (p.text.find(word + ",")) != -1 or (p.text.find(word + "."))  != -1 or (p.text.find(word + " ")) != -1:
                if p.runs[i].text.find(word) != -1:
                    p.runs[i].font.highlight_color = WD_COLOR_INDEX.YELLOW
                    if comment:
                        corresponded_key = find_key_by_value(word,dictionary2)
                        p.runs[i-1].add_comment(f'Suggestion: \"{word}\" is a synonym of dealbreaker word belonging to category {corresponded_key}. We Recommend that you carefully evaluate its usage before considering its reintroduction.',author="DG1")
                        
    return doc



dealbreaker_and_syn_list = combined_check_list + combined_check_list_syn
dealbreaker_and_syn_list = list(set(dealbreaker_and_syn_list))
dealbreaker_and_syn_list

for keyword in dealbreaker_and_syn_list:
    doc=split_Runs(doc,keyword)    

for keyword in dealbreaker_and_syn_list:
    doc=style_Token2(doc,keyword,combined_check_list, dealbreaker_words, synonyms_dealbreaker_words,True)

Giving Comment on Representative and Remedies <br>
if they exist, nothing will happen <br>
if they do not exist, output a warning comment

In [40]:
def check_existence_for_remedy_representative(doc,warning_1 = True, warning_2 = True, warning_3 = True, warning_4 = True):
    if warning_1:
        paragraph1 = doc.add_paragraph()
        comment1 = paragraph1.add_comment("Warning: This document is lack of Representative Category information", author = 'DG1')
    if warning_2:
        paragraph2 = doc.add_paragraph()
        comment2 = paragraph2.add_comment("Warning: This document is lack of Remedy Category information",author = 'DG1')
    if warning_3:
        paragraph3 = doc.add_paragraph()
        comment3 = paragraph3.add_comment("Suggested Warning: This document is lack of synonyms of Representative Category information", author = 'DG1')
    if warning_4:
        paragraph4 = doc.add_paragraph()
        comment4 = paragraph4.add_comment("Suggested Warning: This document is lack of synonyms of Remedy Category information",author = 'DG1')


    return doc



In [41]:
doc = check_existence_for_remedy_representative(doc,representative_warning,remedies_warning,syn_representative_warning, syn_remedies_warning)

Output Path

In [ ]:
##           ###           ##  EEEEEEEEEEEEEEEE             III
 ##         ## ##         ##   E                     III
  ##       ##   ##       ##    E                     III
   ##     ##     ##     ##     EEEEEEEEEEEEEEE              III
    ##   ##       ##   ##      E                     III
     ## ##         ## ##       E                     III
      ##           ##        EEEEEEEEEEEEEEEE             III

In [42]:
# change the path to your current output path
doc.save('/content/drive/MyDrive/Final_pipeline/NDA_db_syno_specialized.docx')

In [ ]:
end_time = time.time()

In [ ]:
total_time = end_time - start_time

In [ ]:
print(f"Total time taken: {total_time:.2f} seconds")